# 9주차 수업

### 1. S&P500 구성 종목 뽑기
### 2.포트폴리오 최적화

**Made by 35기 코딩부장 류제현**

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from tqdm import tqdm # 아마 pip install tqdm 이 필요할지도?

In [ ]:
# pickle 파일: binary 형태로 직렬화 해서 파일을 저장합니다 -> 읽고 쓰는 속도가 CSV보다 매우 빠름 / CSV처럼 직접 수정은 불가능(무조건 파이썬에서만!)
price = pd.read_pickle("Data/stock_adj.pickle")
con = pd.read_pickle("Data/spx_constitute_daily.pickle")

In [ ]:
# 이걸로 매일매일의 S&P500 기업 주식의 수익을 어떻게 뽑아낼 수 있을까요?
con

In [ ]:
# S&P500에 속하면 1, 아니면 NaN을 기록할 빈 데이터프레임을 하나 생성합니다
spx_mask = pd.DataFrame(index=price.index,
                        columns=price.columns)
spx_mask

In [ ]:
# 인덱스를 (매일 날짜)를 따라 반복문을 돌립니다

for date in tqdm(spx_mask.index):
    spx_date = con.loc[con.date == date] # 그 날짜에 해당하는 S&P500 구성종목을 가져옵니다
    corp_list = spx_date.permno # 회사 permno를 가져옵니다
    
    # 위에서 만든 빈 데이터 프레임에 채워줍니다
    spx_mask.loc[date, corp_list] = 1

In [ ]:
# 1이 잘 들어갔네요!
spx_mask

In [ ]:
# S&P500 구성종목 개수 합을 한번 살펴볼까요?
spx_mask.sum(1).plot();

In [ ]:
# S&P500 기업의 리턴을 어떻게 구하냐... 
## 데이터프레임끼리 착 곱해주면 된다
rtn = price.pct_change(fill_method=None) # fill_method=None (NaN 값이 많은 경우 꼭 넣어줘야함 / Pandas 알고리즘이 좀 이상해서 그렇다)
rtn

In [ ]:
# S&P500에 들어간 종목만의 리턴을 계산할 수 있습니다
spx_rtn = spx_mask * rtn
spx_rtn

### 2. 포트폴리오 최적화

In [ ]:
# 데이터를 불러옵니다
etf = pd.read_pickle("ETF.pickle")
etf

In [ ]:
# 리턴을 계산
rtn = etf.pct_change()
rtn

In [ ]:
# 어차피 투자 첫날은 수익이 0이기 때문에, dropna보다는 fillna 0을 해주는게 하루 데이터 손실을 줄일 수 있다
rtn = rtn.fillna(0) 
rtn

In [ ]:
# Rabalancing을 수행할 Range를 정의합니다
start_idx = pd.date_range("2007", "2023", freq="BYS") # 맨 뒤는 포함하지 않습니다
end_idx = pd.date_range("2007", "2023", freq="BY") # 맨 뒤는 포함하지 않습니다

start_idx

In [ ]:
end_idx

In [ ]:
# 목적함수를 정의합니다
##  샤프 최대화 목적함수를 정의한다 (최소화 알고리즘을 쓸 거기 때문에, 샤프지수에 마이너스를 달아준다)

def obj_sharpe(weights, cov_matrix, mean_rtn):
    portfolio_return = np.dot(weights, mean_rtn)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot((cov_matrix), weights)))
    sharpe_ratio = portfolio_return  / portfolio_volatility
    return -sharpe_ratio

In [ ]:
# 최적화 결과를 담을 빈 데이터프레임을 생성
weight_df = pd.DataFrame(columns=rtn.columns) 

In [ ]:
# 최적화를 수행할 제약, Bound와 Initial Weights를 셋팅합니다
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(len(rtn.columns)))
initial_weights = np.ones(len(rtn.columns)) / len(rtn.columns)

In [ ]:
for i in tqdm(range(len(start_idx))):
    start = start_idx[i] 
    end = end_idx[i]
    
    rtn_sample = rtn.loc[start:end]
    
    # 공분산 행렬을 추정합니다
    cov_matrix = rtn_sample.cov()
    mean_rtn = rtn_sample.mean()
    
    result = minimize(obj_sharpe,           # 목적함수 
                      initial_weights,        # 초기 Weight
                      args=(cov_matrix,mean_rtn),   # 목적함수에 들어갈 추가적 인수
                      method='SLSQP',            # Solver
                      constraints= constraints, # 제약식 셋팅   
                      bounds=bounds 
                      )
    
    min_variance_weights = result.x  # 알고리즘이 출력한 최적값 저장
    weight_df.loc[end] = min_variance_weights

In [ ]:
weight_df

In [ ]:
def simulate_strategy(group_weight_df:pd.DataFrame, daily_rtn_df:pd.DataFrame, fee_rate:float):
        '''
        전략의 수익을 평가합니다(Long-Only Portfolio)
        '''
        pf_value = 1
        pf_dict = {}

        weight = group_weight_df.iloc[0] # 시작 weight를 지정해준다 (첫 weight에서 투자 시작, 장마감 직전에 포트폴리오 구성)
        dollar_value = weight * pf_value # Start Dollar Value를 지정
        
        rebalancing_idx = group_weight_df.index # 리벨런싱 할 날들
        start_idx = rebalancing_idx[0]          # 투자 시작일
        
        idx = daily_rtn_df.loc[start_idx:].index
        weight_df = pd.DataFrame(index=idx, columns=daily_rtn_df.columns) # Weight 변화를 기록할 빈 데이터프레임 생성
        weight_df.loc[start_idx] = weight # 시작 weight를 기록

        for idx, row in daily_rtn_df.loc[start_idx:].iloc[1:].iterrows(): # Daily로 반복 / 시작 weight 구성 다음 날부터 성과를 평가
            # 수익률 평가가 리밸런싱보다 선행해야함
            dollar_value = dollar_value * (1+np.nan_to_num(row)) # update the dollar value
            pf_value = np.nansum(dollar_value) # update the pf value
            
            weight = dollar_value / pf_value # update the weight

            if idx in rebalancing_idx: # Rebalancing Date (장마감 직전에 리벨런싱 실시)
                weight = group_weight_df.loc[idx] # Weight Rebalancing
                target_dollar_value = np.nan_to_num(pf_value * weight) * (1 - fee_rate)
                dollar_fee = np.nansum(np.abs(target_dollar_value - np.nan_to_num(dollar_value)) * fee_rate) # fee계산
                pf_value = pf_value - dollar_fee # fee 차감

                dollar_value = weight * pf_value  # dollar value를 Rebalancing 이후로 update
            
            weight_df.loc[idx] = weight # weight 변화를 기록
            pf_dict[idx] = pf_value

        return pf_dict

In [ ]:
result = simulate_strategy(weight_df, rtn, fee_rate=0)

In [ ]:
pd.Series(result).plot(figsize=(24,12));

    한 학기 고생 많으셨습니다!